#### Setup

In [2]:
import sys
import numpy as np
import pandas as pd
import math
import random
from concorde.tsp import TSPSolver
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
print(sys.version)

3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 22:45:16) 
[Clang 9.0.1 ]


In [3]:
# GLOBAL VARIABLES
field_width = 100
field_height = 100
depot_x = 50
depot_y = 50

In [171]:
class Instance():
    
    def __init__(self, xlocs, ylocs, demands, solve_TSP=True):

        self.size = len(demands)-1
        self.demands = demands
        self.xlocs = xlocs
        self.ylocs = ylocs
        self.distances = self.calc_distance_matrix()
        self.tour = 'None'
        if solve_TSP:
            self.tour = self.solve_TSP()
        
    def calc_distance_matrix(self):

        distances = np.zeros((self.size+1, self.size+1), dtype=float)
        for i in range(self.size+1):
            for j in range(self.size+1):
                new_dist = math.sqrt((self.xlocs[i]-self.xlocs[j])**2 + (self.ylocs[i]-self.ylocs[j])**2)
                distances[i,j] = new_dist
        return distances

    def get_lowerbound(self, capacity):
        return (2/capacity) * sum([self.demands[i]*self.distances[0,i]
                                        for i in range(len(self.demands))])
    
    def get_fleet_size(self, route_size):
        assert self.size % route_size == 0, "Number of customers must be evenly divisible by the route size."
        return int(self.size / route_size)
    
    def solve_TSP(self):
        solver = TSPSolver.from_data(inst.xlocs, inst.ylocs, 'EUC_2D')
        solution = solver.solve()
        return solution.tour


In [82]:
def solve_TSP(inst):
    
    solver = TSPSolver.from_data(inst.xlocs, inst.ylocs, 'EUC_2D')
    solution = solver.solve()
    return solution

def get_dedicated_routes(inst, route_size):
    
    tour =  inst.tour[1:]
    routes = []
    for i in range(0,len(tour),route_size):
        new_route = tour[i:i+route_size]
        routes.append(new_route)
    return routes


In [7]:
def get_circular_cost(inst,segment):
    return sum([inst.distances[segment[i],segment[i+1]] for i in range(len(segment)-1)])

def get_radial_cost(inst,segment):
    """Assumes vehicle travels to/from the depot at segment endpoints."""
    return inst.distances[0,segment[0]] + inst.distances[0,segment[-1]]

In [139]:
def pack_vehicles(inst, capacity):
    
    segments = []
    i = 0
    new_seg = {}
    while i < len(inst.tour):
        cust = inst.tour[i]
        for d in range(1,inst.demands[cust]+1):
            if cust not in new_seg:
                new_seg[cust] = 1
            else:
                new_seg[cust]+=1
            if sum(new_seg.values()) == capacity:
                seg_array = np.array(list(new_seg))
                segments.append(seg_array)
                new_seg = {}
        i+=1
    seg_array = np.array(list(new_seg))
    segments.append(seg_array)
    return segments

In [140]:
pack_vehicles(inst, capacity)

[array([4, 1, 9, 2, 7], dtype=int32), array([7, 6, 8, 3], dtype=int32)]

In [142]:
def solve_fully_flexible(inst, capacity):
    
    # Create list of customer routes
    segments = pack_vehicles(inst, capacity)     
    
    # Get route costs
    radial_costs = [get_radial_cost(inst,seg) for seg in segments]
    circular_costs = [get_circular_cost(inst,seg) for seg in segments]
    
    # Return total circular cost, total radial cost, and number of trips needed
    return sum(circular_costs), sum(radial_costs), len(segments)

In [174]:
N = 10
dmin = 0
dmax = 8
cust_x = field_width*np.random.random(N)
cust_y = field_height*np.random.random(N)
cust_dems = np.random.randint(dmin,dmax,N)
xlocs = np.append([depot_x], cust_x)
ylocs = np.append([depot_y], cust_y)
demands = np.append([0], cust_dems)
inst = Instance(xlocs, ylocs, demands)

In [53]:
capacity=20
inst.get_lowerbound(capacity)

158.970585216701

In [175]:
#inst.solve_TSP()
inst.tour

array([ 0, 10,  1,  9,  4,  6,  2,  7,  8,  3,  5], dtype=int32)

In [87]:
#tsp = solve_TSP(inst)
inst.solve_TSP()
routes = get_dedicated_routes(inst, route_size)
print('Big tour:', inst.tour)
print('Routes:', routes)
print('Radial costs:', [get_radial_cost(inst,seg).round(1) for seg in routes])
print('Circular costs:', [get_circular_cost(inst,seg).round(1) for seg in routes])

Big tour: [ 0  5  6  4  1  9  8  7  2 10  3]
Routes: [array([5, 6, 4, 1, 9], dtype=int32), array([ 8,  7,  2, 10,  3], dtype=int32)]
Radial costs: [69.3, 54.5]
Circular costs: [42.8, 93.9]
